In [62]:
# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introducción

En la parte 1 de este desafío, se realizó el procesamiento previo y el entrenamiento de un modelo a partir de un conjunto de datos base proporcionados. En este segundo paso, se integrará todas las transformaciones y eventos de entrenamiento creados previamente en un Pipeline completo para *deploy* en **Watson Machine Learning**.

### Preparación del Notebook

Primero realizaremos la instalación do scikit-learn y la importación de las mismas bibliotecas utilizadas anteriormente

In [63]:
!pip install scikit-learn==0.20.0 --upgrade

Requirement already up-to-date: scikit-learn==0.20.0 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.20.0)


In [64]:
import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

Es necesario volver a insertar el conjunto de datos base como un pandas dataframe, siguiendo las instrucciones

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Después de seleccionar la opción **"Insert to code"**, la celda de abajo se llenará con el código necesario para importar y leer los datos en el archivo .csv como un Pandas DataFrame.

In [65]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_90a2289545db4e3eac3255f8240fbad9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RueLUmqCx740mc88R7IOOmCjjPRXIXmdSlIYLcyYQY9X',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_90a2289545db4e3eac3255f8240fbad9.get_object(Bucket='desafio2ibm-donotdelete-pr-oukbdjkuqn057r',Key='dataset-tortuga-desafio-2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Unnamed: 0,NAME,USER_ID,HOURS_DATASCIENCE,HOURS_BACKEND,HOURS_FRONTEND,NUM_COURSES_BEGINNER_DATASCIENCE,NUM_COURSES_BEGINNER_BACKEND,NUM_COURSES_BEGINNER_FRONTEND,NUM_COURSES_ADVANCED_DATASCIENCE,NUM_COURSES_ADVANCED_BACKEND,NUM_COURSES_ADVANCED_FRONTEND,AVG_SCORE_DATASCIENCE,AVG_SCORE_BACKEND,AVG_SCORE_FRONTEND,PROFILE
0,28,Stormy Muto,58283940,7.0,39.0,29.0,2.0,4.0,0.0,2.0,5.0,0.0,84.0,74.0,NaN,beginner_front_end
1,81,Carlos Ferro,1357218,32.0,0.0,44.0,2.0,0.0,0.0,0.0,5.0,0.0,67.0,45.0,NaN,beginner_front_end
2,89,Robby Constantini,63212105,45.0,0.0,59.0,0.0,5.0,4.0,0.0,4.0,1.0,NaN,54.0,47.0,advanced_front_end
3,138,Paul Mckenny,23239851,36.0,19.0,28.0,0.0,5.0,7.0,0.0,5.0,3.0,NaN,71.0,89.0,beginner_data_science
4,143,Jean Webb,72234478,61.0,78.0,38.0,6.0,11.0,0.0,4.0,3.0,0.0,66.0,85.0,NaN,advanced_front_end


### Construcción del Pipeline completo para el encapsulamiento en WML

#### Preparando transformaciones personalizadas para cargar en WML

En el paso anterior, se mostró cómo crear una transformación personalizada, declarando una clase Python con los métodos ``fit`` y ``transform``.

    - Código de transformación personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primero copiamos el dataframe de entrada 'X' de entrada
            data = X.copy()
            # Devolvemos un nuevo marco de datos sin las columnas no deseadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar estos tipos de transformaciones personalizadas con Pipelines en Watson Machine Learning, primero debe empaquetar su código personalizado como una biblioteca de Python. Esto se puede hacer fácilmente usando la herramienta *setuptools*.

En el siguiente repositorio de git: https://github.com/vnderlev/sklearn_transforms tenemos todos los archivos necesarios para crear un paquete de Python, llamado **my_custom_sklearn_transforms**.
Este paquete tiene la siguiente estructura de archivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
El archivo principal, que contendrá el código para nuestras transformaciones personalizadas, es el archivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Si accedes a él en el repositorio, notarás que contiene exactamente el mismo código declarado en el primer paso (la clase DropColumns).

Si has declarado sus propias transformaciones (además de la DropColumn proporcionada), debes agregar todas las clases de esas transformaciones creadas en este mismo archivo. Para hacer esto, debes hacer fork de este repositorio (esto se puede hacer en la propia interfaz web de Github, haciendo clic en el botón como se muestra en la imagen a continuación) y agregue sus clases personalizadas al archivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/2lZ4Ty2.png "forking-a-repo")

Si solo hizo uso de la transformación proporcionada (DropColumns), puede omitir este paso de fork y continuar usando el paquete base provisto. :)

Después de preparar su paquete de Python con sus transformaciones personalizadas, reemplace el enlace del repositorio de git en la celda a continuación y ejecútelo. Si no ha preparado ninguna transformación nueva, ejecute la celda con el enlace del repositorio ya proporcionado.

<hr>
    
**OBSERVACIÓN**

Si la ejecución de la celda a continuación devuelve un error de que el repositorio ya existe, ejecute:

**!rm -r -f sklearn_transforms**

In [66]:
## reemplace el enlace a continuación con el enlace de su repositorio de git (si corresponde)
!git clone https://github.com/guillermo-lr/sklearn_transforms.git

fatal: destination path 'sklearn_transforms' already exists and is not an empty directory.


In [67]:
!cd sklearn_transforms
!ls -ltr

total 84
drwxr-x--- 5 dsxuser dsxuser  4096 Aug 16 22:11 sklearn_transforms
-rw-r----- 1 dsxuser dsxuser 78558 Aug 16 22:17 sklearn_transforms.zip


Para cargar el código en WML, necesitamos enviar un archivo .zip con todo el código fuente, luego comprimiremos el directorio clonado a continuación:

In [68]:
!zip -r sklearn_transforms.zip sklearn_transforms

updating: sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/setup.py (deflated 46%)
updating: sklearn_transforms/.git/ (stored 0%)
updating: sklearn_transforms/.git/refs/ (stored 0%)
updating: sklearn_transforms/.git/refs/tags/ (stored 0%)
updating: sklearn_transforms/.git/refs/remotes/ (stored 0%)
updating: sklearn_transforms/.git/refs/remotes/origin/ (stored 0%)
updating: sklearn_transforms/.git/refs/remotes/origin/HEAD (stored 0%)
updating: sklearn_transforms/.git/refs/heads/ (stored 0%)
updating: sklearn_transforms/.git/refs/heads/master (stored 0%)
updating: sklearn_transforms/.git/branches/ (stored 0%)
updating: sklearn_transforms/.git/hooks/ (stored 0%)
updating: sklearn_transforms/.git/hooks/commit-msg.sample (deflated 44%)
updating: sklearn_transforms/.git/hooks/pre-rebase.sample (deflated 59%)
updating: sklearn_transforms/.git/hooks/prepare-commit-msg.sample (deflated 46%)
updating: sklearn_transforms/.git/hooks/pre-applypatch.sample (deflated 36%)
updating: sklear

Con el archivo zip de nuestro paquete cargado en el Kernel de este notebook, podemos usar la herramienta pip para instalarlo, de acuerdo con la celda a continuación:

In [69]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Stored in directory: /home/dsxuser/.tmp/pip-ephem-wheel-cache-lppnk4gb/wheels/8f/88/32/f886e7510a37b111e2a1b7e689e04450acda46732970a7ed78
Successfully built my-custom-sklearn-transforms
  Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


¡Ahora podemos importar nuestro paquete personalizado a nuestro notebook!

Importaremos la transformación DropColumns. Si tienes otras transformaciones personalizadas, ¡no olvides importarlas!

In [70]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns, Scalador


#### Declarando un Pipeline

Después de importar transformaciones personalizadas como un paquete de Python, podemos proceder a la declaración de nuestro Pipeline.

El proceso es muy similar al realizado en la primera etapa, pero con algunas diferencias importantes, ¡así que presta mucha atención!

El Pipeline de ejemplo tiene tres etapas: 

    - Remover las colunas "NAME" e "Unnamed: 0"
    - Asignar "ceros" a todos los valores faltantes
    - insertar datos preprocesados como entrada en un modelo entrenado
    
Recordando, la entrada de este Pipeline será el conjunto de datos brutos proporcionados, excepto la columna "PROFILE" (variable de objetivo que será determinada por el modelo).

Entonces tendremos 16 valores de entrada en el **PIPELINE** (en el modelo habrá 14 entradas, ya que las columnas "NAME" y "Unnamed: 0" se eliminarán en la primera etapa después de la transformación DropColumns).


    Unnamed: 0                          - Esta columna no tiene nombre y debe ser eliminada del dataset
    NAME                                - Nombre del estudiante
    USER_ID                             - Número de identificación del estudiante
    HOURS_DATASCIENCE                   - Número de horas de estudio en Data Science
    HOURS_BACKEND                       - Número de horas de estudio en Web (Back-End)
    HOURS_FRONTEND                      - Número de horas de estudio en Web (Front-End)
    NUM_COURSES_BEGINNER_DATASCIENCE    - Número de cursos de nivel principiante en Data Science completados por el estudiante
    NUM_COURSES_BEGINNER_BACKEND        - Número de cursos de nivel principiante en Web (Back-End) completados por el estudiante
    NUM_COURSES_BEGINNER_FRONTEND       - Número de cursos de nivel principiante en Web (Front-End) completados por el estudiante
    NUM_COURSES_ADVANCED_DATASCIENCE    - Número de cursos de nivel avanzado en Data Science completados por el estudiante
    NUM_COURSES_ADVANCED_BACKEND        - Número de cursos de nivel avanzado en Web (Back-End) completados por el estudiante
    NUM_COURSES_ADVANCED_FRONTEND       - Número de cursos de nivel avanzado en Web (Front-End) completados por el estudiante
    AVG_SCORE_DATASCIENCE               - Promedio acumulado en cursos de Data Science completados por el estudiante
    AVG_SCORE_BACKEND                   - Promedio acumulado en cursos de Web (Back-End) completados por el estudiante
    AVG_SCORE_FRONTEND                  - Promedio acumulado en cursos de Web (Front-End) completados por el estudiante

La salida del Pipeline será un valor estimado para la columna "PROFILE".

In [71]:
# Crear una transformación personalizada ``DropColumns``

rm_columns = DropColumns(
    columns=["NAME", "Unnamed: 0"]
)

In [72]:
# Crear un objeto ``SimpleImputer``

si = SimpleImputer(
    missing_values=np.nan,  # los valores que faltan son del tipo ``np.nan`` (Pandas estándar)
    strategy='constant',  # la estrategia elegida es cambiar el valor faltante por una constante
    fill_value=0,  # la constante que se usará para completar los valores faltantes es un int64 = 0
    verbose=0,
    copy=True
)

In [73]:
# si.fit(df_data_1)
# df_data_1 = si.transform(df_data_1)

In [74]:
sc = Scalador()


In [75]:
# Definicion de columnas que seran features
features = [
    "Unnamed: 0", "NAME", "USER_ID", "HOURS_DATASCIENCE", "HOURS_BACKEND", "HOURS_FRONTEND",
    "NUM_COURSES_BEGINNER_DATASCIENCE", "NUM_COURSES_BEGINNER_BACKEND", "NUM_COURSES_BEGINNER_FRONTEND",
    "NUM_COURSES_ADVANCED_DATASCIENCE", "NUM_COURSES_ADVANCED_BACKEND", "NUM_COURSES_ADVANCED_FRONTEND",
    "AVG_SCORE_DATASCIENCE", "AVG_SCORE_BACKEND", "AVG_SCORE_FRONTEND"
]

# Definición de variable objetico
target = ["PROFILE"]

# Preparación de argumentos para los métodos de la biblioteca ``scikit-learn``
X = df_data_1[features]
y = df_data_1[target]

**¡¡ATENCIÓN!!**

La celda de arriba, aunque muy similar a la definición de características en la primera etapa de este desafío, ¡tiene una gran diferencia!

Contiene las columnas "NAME" y "Unnamed: 0" como features. Esto se debe a que en este caso estas son las entradas *PIPELINE*, no el modelo.

In [76]:
# Separación de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

En la celda a continuación, se declara un objeto scikit-learn **Pipeline**, donde se declara el parámetro *steps*, que no es más que una lista de los pasos en nuestro pipeline:

    'remove_cols'     - transformación personalizada DropColumns
    'imputer'         - transformación scikit-learn incorporada para asignar valores faltantes
    'dtc'             - un clasificador a través del árbol de decisión

Tenga en cuenta que pasamos como transformaciones instanciadas anteriormente, bajo el nombre `rm_columns` y` si`.

In [77]:
# Creación de nuestro pipeline para almacenamiento en Watson Machine Learning:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('scalae', sc),
        ('imputer', si),
        ('dtc', QuadraticDiscriminantAnalysis()),
    ]
)

Luego, ejecutaremos el método `fit ()` de Pipeline, realizando el preprocesamiento y entrenamiento del modelo a la vez.

In [78]:
# Inicialización de Pipeline (preprocesamiento y formación de modelos)
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('remove_cols', DropColumns(columns=['NAME', 'Unnamed: 0'])), ('scalae', Scalador()), ('imputer', SimpleImputer(copy=True, fill_value=0, missing_values=nan,
       strategy='constant', verbose=0)), ('dtc', QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001))])

Ahora que tenemos un pipeline completo, con los pasos de preprocesamiento configurados y también un modelo por árbol de decisiones ya entrenado, ¡podemos integrarnos con Watson Machine Learning!

### Encapsulación de un Pipeline personalizado en Watson Machine Learning 

#### Establecer una conexión entre el cliente WML Python y su instancia de servicio en la nube

In [79]:
# Biblioteca de Python con implementación de un cliente HTTP para la API de WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Las siguientes celdas desplegarán el pipeline declarado en este notebook en WML. Continúe solo si ya está satisfecho con su modelo y cree que es hora de implementar su solución.

Pegue las credenciales de su instancia de Watson Machine Learning en la variable de la celda siguiente.

Es importante que la variable que contiene los valores tenga el nombre de `` wml_credentials`` para que las siguientes celdas de este notebook se ejecuten correctamente.

In [80]:
wml_credentials = {
  "apikey": "LstA8scuoeAOzQcvPtz73srObREVbVt7VuOH1nikFz-P",
  "iam_apikey_description": "Auto-generated for key d292a085-df0d-47ff-8f9b-a51c2aa3f670",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/5c6c6ce2b75c472aaa969e8c719fbfa7::serviceid:ServiceId-6760e47d-2c96-4adf-a4b3-001edae30f52",
  "instance_id": "177cec46-530a-43c5-a120-491225cdd9aa",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [81]:
# Creación de un objeto cliente de Watson Machine Learning a partir de las credenciales proporcionadas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [82]:
# Extracción de detalles de su instancia de Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/177cec46-530a-43c5-a120-491225cdd9aa/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 2,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 2,
         

**¡¡ATENCIÓN!!**

¡Preste atención a los límites de consumo de su instancia de Watson Machine Learning!

Si expira la capa libre, no podrá evaluar su modelo (¡ya que es necesario realizar algunas llamadas a API que consumen predicciones!)

#### Listado de todos los artefactos almacenados en su WML

Para listar todos los artefactos almacenados en Watson Machine Learning, puede utilizar la siguiente función:

    clientWML.repository.list()

In [83]:
# Listado de todos los artefactos almacenados en su WML

clientWML.repository.list()

------------------------------------  ----------------------------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                                                  CREATED                   FRAMEWORK          TYPE
62f9461b-d9bb-4a4c-8fd7-5e544460f2f8  desafio-2-mbtc2020-pipeline-es-2                      2020-08-16T22:13:37.581Z  scikit-learn-0.20  model
c59047f4-3714-4ec0-8fb2-efb5366a7b6a  desafio-2-mbtc2020-pipeline-es-2                      2020-08-16T16:55:48.852Z  scikit-learn-0.20  model
71eb9f8f-4ca4-4642-9d3c-c1bc9b747e99  desafio-2-mbtc2020-deployment-es-2                    2020-08-16T22:13:41.219Z  scikit-learn-0.20  online deployment
530768c7-4df5-4bba-ab47-137d6226dbfb  desafio-2-mbtc2020-deployment-es-2                    2020-08-16T16:57:54.119Z  scikit-learn-0.20  online deployment
be01f033-fde3-4455-b945-fd4353512571  my_custom_sklearn_transform_es_2_guillermo_rodriguez  2020-08-16T22:1

En el plan LITE de Watson Machine Learning, solo se puede implementar un modelo a la vez. Si ya tiene un modelo en línea en su instancia, puede eliminarlo usando el método clientWML.repository.delete ():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Crear una nueva definición de paquete Python personalizada en WML

El primer paso para realizar su implementación es almacenar el código de las transformaciones personalizadas creadas.

Para este paso solo necesitamos el archivo .zip del paquete creado (¡que ya hemos cargado en el Kernel!)

In [84]:
# Definición de metadatos de nuestro paquete con transformaciones personalizadas 
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_es_2_guillermo_rodriguez_code",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip creado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefactos en tiempo de ejecución almacenados en WML:")
clientWML.repository.list()


 Lista de artefactos en tiempo de ejecución almacenados en WML:
------------------------------------  ---------------------------------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                                                       CREATED                   FRAMEWORK          TYPE
62f9461b-d9bb-4a4c-8fd7-5e544460f2f8  desafio-2-mbtc2020-pipeline-es-2                           2020-08-16T22:13:37.581Z  scikit-learn-0.20  model
c59047f4-3714-4ec0-8fb2-efb5366a7b6a  desafio-2-mbtc2020-pipeline-es-2                           2020-08-16T16:55:48.852Z  scikit-learn-0.20  model
71eb9f8f-4ca4-4642-9d3c-c1bc9b747e99  desafio-2-mbtc2020-deployment-es-2                         2020-08-16T22:13:41.219Z  scikit-learn-0.20  online deployment
530768c7-4df5-4bba-ab47-137d6226dbfb  desafio-2-mbtc2020-deployment-es-2                         2020-08-16T16:57:54.119Z  scikit-learn-0.20  online deployment
70b6f44d-526

#### Creación de una nueva definición personalizada de runtime Python en WML

El segundo paso es almacenar una definición de runtime Python para usar nuestra biblioteca personalizada.

Esto puede hacerse de la siguiente manera:

In [85]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_es_2",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalles de runtime almacenados:")
print(json.dumps(runtime_details, indent=4))


 Detalles de runtime almacenados:
{
    "metadata": {
        "guid": "1a9d41df-69a3-4ac6-859b-3bd4f449c360",
        "url": "https://us-south.ml.cloud.ibm.com/v4/runtimes/1a9d41df-69a3-4ac6-859b-3bd4f449c360",
        "created_at": "2020-08-16T22:18:38.126Z"
    },
    "entity": {
        "name": "my_custom_wml_runtime_es_2",
        "description": "A Python runtime with custom sklearn Transforms",
        "custom_libraries": [
            {
                "name": "my_custom_sklearn_transform_es_2_guillermo_rodriguez_code",
                "version": "1.0",
                "url": "https://private.us-south.ml.cloud.ibm.com/v4/libraries/70b6f44d-5262-4470-8c20-858b932787de"
            }
        ],
        "content_url": "https://private.us-south.ml.cloud.ibm.com/v4/runtimes/1a9d41df-69a3-4ac6-859b-3bd4f449c360/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    }
}


In [86]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

------------------------------------  --------------------------  ------------------------  ----------
GUID                                  NAME                        CREATED                   PLATFORM
1a9d41df-69a3-4ac6-859b-3bd4f449c360  my_custom_wml_runtime_es_2  2020-08-16T22:18:38.126Z  python-3.6
60dc1e15-760c-421d-b40b-ee98c0fb9da5  my_custom_wml_runtime_es_2  2020-08-16T22:13:36.122Z  python-3.6
27dde2a2-f25b-43cc-8f1b-c722ee7fcaa6  my_custom_wml_runtime_es_2  2020-08-16T16:29:45.181Z  python-3.6
------------------------------------  --------------------------  ------------------------  ----------


#### Crear una nueva definición de Pipeline personalizado en WML

Finalmente crearemos una definición (metadatos) para que nuestro Pipeline se aloje en WML.

Definimos como parámetros un nombre para el artefacto y el ID de runtime creado anteriormente.

In [87]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-es-2',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Luego llamamos al método para almacenar la nueva definición:

In [88]:
# Función para almacenar una definición de Pipeline en WML 
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` es la variable creada previamente y contiene nuestro Pipeline ya entrenado :)
    meta_props=model_meta,  # Metadatos definidos en la celda anterior
    training_data=None  # No cambie este parámetro
)

print("\n Lista de artefactos almacenados en WML:")
clientWML.repository.list()

# Detalles del modelo alojado en Watson Machine Learning
print("\n Metadatos almacenados del modelo:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefactos almacenados en WML:
------------------------------------  ---------------------------------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                                                       CREATED                   FRAMEWORK          TYPE
2552b236-ad52-416a-837a-6a9ac4064bda  desafio-2-mbtc2020-pipeline-es-2                           2020-08-16T22:18:39.642Z  scikit-learn-0.20  model
62f9461b-d9bb-4a4c-8fd7-5e544460f2f8  desafio-2-mbtc2020-pipeline-es-2                           2020-08-16T22:13:37.581Z  scikit-learn-0.20  model
c59047f4-3714-4ec0-8fb2-efb5366a7b6a  desafio-2-mbtc2020-pipeline-es-2                           2020-08-16T16:55:48.852Z  scikit-learn-0.20  model
71eb9f8f-4ca4-4642-9d3c-c1bc9b747e99  desafio-2-mbtc2020-deployment-es-2                         2020-08-16T22:13:41.219Z  scikit-learn-0.20  online deployment
530768c7-4df5-4bba-ab47-137d6226dbfb  desafio-2

#### Despliegue de su modelo para consumo inmediato por otras aplicaciones

In [89]:
# El modelo finalmente se implementa usando el método `` deployments.create () ``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # No cambie este parámetro
    name="desafio-2-mbtc2020-deployment-es-2",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # No cambie este parámetro
    deployment_type='online',  # No cambie este parámetro
    deployment_format='Core ML',  # No cambie este parámetro
    meta_props=model_meta  # No cambie este parámetro
)



#######################################################################################

Synchronous deployment creation for uid: '2552b236-ad52-416a-837a-6a9ac4064bda' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS..
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b454e0a1-06d8-4264-beef-5cdde9282bf3'
------------------------------------------------------------------------------------------------




#### Prueba de un modelo alojado en Watson Machine Learning

In [90]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("Su URL de llamada a la API es: {}".format(model_endpoint_url))

Su URL de llamada a la API es: https://us-south.ml.cloud.ibm.com/v3/wml_instances/177cec46-530a-43c5-a120-491225cdd9aa/deployments/b454e0a1-06d8-4264-beef-5cdde9282bf3/online


In [91]:
# Detalles del despliegue realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # este es su ID de implementación!
)

print("Metadatos de despliegue realizado: \n")
print(json.dumps(deployment_details, indent=4))

Metadatos de despliegue realizado: 

{
    "metadata": {
        "guid": "b454e0a1-06d8-4264-beef-5cdde9282bf3",
        "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/177cec46-530a-43c5-a120-491225cdd9aa/deployments/b454e0a1-06d8-4264-beef-5cdde9282bf3",
        "created_at": "2020-08-16T22:18:42.900Z",
        "modified_at": "2020-08-16T22:18:43.397Z"
    },
    "entity": {
        "runtime_environment": "python-3.6",
        "name": "desafio-2-mbtc2020-deployment-es-2",
        "scoring_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/177cec46-530a-43c5-a120-491225cdd9aa/deployments/b454e0a1-06d8-4264-beef-5cdde9282bf3/online",
        "deployable_asset": {
            "name": "desafio-2-mbtc2020-pipeline-es-2",
            "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/177cec46-530a-43c5-a120-491225cdd9aa/published_models/2552b236-ad52-416a-837a-6a9ac4064bda",
            "guid": "2552b236-ad52-416a-837a-6a9ac4064bda",
            "description": "my

In [92]:
scoring_payload = {
    'fields': [
        "Unnamed: 0", "NAME", "USER_ID", "HOURS_DATASCIENCE", "HOURS_BACKEND", "HOURS_FRONTEND",
        "NUM_COURSES_BEGINNER_DATASCIENCE", "NUM_COURSES_BEGINNER_BACKEND", "NUM_COURSES_BEGINNER_FRONTEND",
        "NUM_COURSES_ADVANCED_DATASCIENCE", "NUM_COURSES_ADVANCED_BACKEND", "NUM_COURSES_ADVANCED_FRONTEND",
        "AVG_SCORE_DATASCIENCE", "AVG_SCORE_BACKEND", "AVG_SCORE_FRONTEND"
    ],
    'values': [
        [
            0,"Paula Waters",85123728,0.0,0.0,86.0,72.0,42.0,0.0,0.0,26.0,184.0,37.0,63.0,38.0,
        ]
    ]
}

# Definición de la variable objetivo
target = ["PROFILE"]

print("\n Payload de datos para clasificar:")
print(json.dumps(scoring_payload, indent=4))


 Payload de datos para clasificar:
{
    "fields": [
        "Unnamed: 0",
        "NAME",
        "USER_ID",
        "HOURS_DATASCIENCE",
        "HOURS_BACKEND",
        "HOURS_FRONTEND",
        "NUM_COURSES_BEGINNER_DATASCIENCE",
        "NUM_COURSES_BEGINNER_BACKEND",
        "NUM_COURSES_BEGINNER_FRONTEND",
        "NUM_COURSES_ADVANCED_DATASCIENCE",
        "NUM_COURSES_ADVANCED_BACKEND",
        "NUM_COURSES_ADVANCED_FRONTEND",
        "AVG_SCORE_DATASCIENCE",
        "AVG_SCORE_BACKEND",
        "AVG_SCORE_FRONTEND"
    ],
    "values": [
        [
            0,
            "Paula Waters",
            85123728,
            0.0,
            0.0,
            86.0,
            72.0,
            42.0,
            0.0,
            0.0,
            26.0,
            184.0,
            37.0,
            63.0,
            38.0
        ]
    ]
}


In [93]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))


 Resultados:
{
    "fields": [
        "prediction",
        "probability"
    ],
    "values": [
        [
            "beginner_backend",
            [
                0.0012497779202254242,
                0.03809139531242915,
                0.1944838194079779,
                0.5904812452043887,
                0.17475907371419067,
                0.0009346884407881788
            ]
        ]
    ]
}


<hr>

## ¡Felicidades! 

Si todo salió bien, ¡ya tiene un clasificador basado en aprendizaje automático encapsulado como una API REST!

Para probar tu solución integrada con un asistente virtual y realizar el envío, visita la página:

https://tortuga.maratona.dev

Necesitarás la URL del endpoint de tu modelo y las credenciales WML :)